# 경로 설정

In [1]:
import os
current_directory = os.getcwd()
# os.chdir('../'), os.chdir('./python')
print(f"cncurrent_directory : {current_directory}")

cncurrent_directory : c:\Users\su1jun\OneDrive - 서울과학기술대학교\Capstone\Capstone\Test


# 데이터 모델 구축 (학습 > 훈련 > 검증)

In [ ]:
from ultralytics import YOLO

In [ ]:
model_path = os.path.join(current_directory, 'yolov8m.pt')
if os.path.exists(model_path):
    model = YOLO(model_path) # pass any model type
else:
    print("Not defined file")

# results = model.train(epochs=5)
# model.val()  # It'll automatically evaluate the data you trained.

In [5]:
data_path = os.path.join(current_directory, 'datasets', 'SKU-110K.yaml')
if os.path.exists(data_path):
    model.train(data=data_path, epochs=100)
else:
    print("Not defined file")

Ultralytics YOLOv8.0.182  Python-3.11.4 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: task=detect, mode=train, model=c:\Users\qaxsd\OneDrive - \Capstone\yolov8m.pt, data=c:\Users\qaxsd\OneDrive - \Capstone\datasets\SKU-110K.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False,

In [ ]:
# Tune hyperparameters
model.tune(data='coco8.yaml', epochs=30, iterations=300, optimizer='AdamW', plots=False, save=False, val=False)

### Validation

In [5]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each categorya

Ultralytics YOLOv8.0.182  Python-3.11.4 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients

Dataset 'coco.yaml' images not found , missing path 'C:\Users\qaxsd\OneDrive - \datasets\coco\val2017.txt'
100%|██████████| 169M/169M [00:48<00:00, 3.64MB/s] 
Unzipping C:\Users\qaxsd\OneDrive - 서울과학기술대학교\datasets\coco2017labels-segments.zip to C:\Users\qaxsd\OneDrive - 서울과학기술대학교\datasets\coco...: 100%|██████████| 122232/122232 [02:26<00:00, 831.83file/s] 


# 입력값 예측

In [4]:
import cv2

cap = cv2.VideoCapture(1)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.namedWindow("YOLOv8 Inference", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("YOLOv8 Inference", 800, 600)  # 윈도우의 너비와 높이를 800x600으로 설정
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 441.8ms
Speed: 2.7ms preprocess, 441.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 342.4ms
Speed: 4.4ms preprocess, 342.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 318.8ms
Speed: 0.0ms preprocess, 318.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 364.2ms
Speed: 1.0ms preprocess, 364.2ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 428.3ms
Speed: 0.0ms preprocess, 428.3ms inference, 8.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 379.1ms
Speed: 1.7ms preprocess, 379.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 350.9ms
Speed: 0.0ms preprocess, 350.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 354.6ms
Speed: 0.0ms prepr

In [ ]:
# Process results generator
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs

# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    im.save('results.jpg')  # save image